In [ ]:
# Installments of required tables
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#  BigQuery API activation
from google.colab import auth
auth.authenticate_user()

In [ ]:
# big query add-on installation

!pip install --quiet google-cloud-bigquery
from google.cloud import bigquery

In [ ]:
# BigQuery client initiation
client = bigquery.Client(project="homecredit-478707")

# From BigQuery
query = """
SELECT *
FROM `homecredit-478707.Homecredit_Tables.POS_CASH_Balance`
"""
POS_CASH_Balance = client.query(query).to_dataframe()

In [ ]:
POS_CASH_Balance.info()

# Primary Key Check


In [ ]:
duplicates = POS_CASH_Balance.duplicated(
    subset=["SK_ID_PREV", "MONTHS_BALANCE"]
).sum()

print("Duplicate PK rows:", duplicates)
print("Total rows:", POS_CASH_Balance.shape[0])



Primary Key Explanation
Primary Key Validation – POS_CASH_Balance

The natural primary key of the POS_CASH_Balance dataset is:

(SK_ID_PREV + MONTHS_BALANCE)
Why this key is unique

SK_ID_PREV identifies a single POS loan account.

MONTHS_BALANCE represents monthly snapshots of that loan, recorded as negative integers (e.g., -1, -2, -3, …), where each value corresponds to one month in the past.

Together, these two fields uniquely define one specific month of activity for one POS loan.

# Feature Engineering

STEP 1 – Create monthly-level behavioral features

POS_CASH_Balance: one row per month per POS loan (SK_ID_PREV)

Goal: extract meaningful POS loan behavior signals at month level

In [ ]:
# 1.1 Delinquency flags
POS_CASH_Balance["POS_LATE_MONTH_FLAG"] = (POS_CASH_Balance["SK_DPD"] > 0).astype(int)
# 1 if the POS loan is late in that month, 0 otherwise

POS_CASH_Balance["POS_SERIOUS_LATE_MONTH_FLAG"] = (POS_CASH_Balance["SK_DPD_DEF"] > 0).astype(int)
# 1 if the POS loan is seriously delinquent that month (default-level delay)

In [ ]:
# 1.2 Instalment progress features
POS_CASH_Balance["POS_INSTALMENTS_PAID"] = (
    POS_CASH_Balance["CNT_INSTALMENT"] - POS_CASH_Balance["CNT_INSTALMENT_FUTURE"]
)
# Number of instalments already paid for that POS loan at this snapshot

# Handle division by zero safely
POS_CASH_Balance["POS_REMAINING_RATIO"] = (
    POS_CASH_Balance["CNT_INSTALMENT_FUTURE"] / (POS_CASH_Balance["CNT_INSTALMENT"] + 1e-6)
)
# Remaining instalments as a fraction of total instalments (how much of the loan is still open)

POS_CASH_Balance["POS_PROGRESS_RATIO"] = (
    POS_CASH_Balance["POS_INSTALMENTS_PAID"] / (POS_CASH_Balance["CNT_INSTALMENT"] + 1e-6)
)
# Progress through the instalment schedule (how much of the loan is already paid)

In [ ]:
# 1.3 Contract status flags (Active vs Completed vs others)

POS_CASH_Balance["POS_ACTIVE_FLAG"] = (POS_CASH_Balance["NAME_CONTRACT_STATUS"] == "Active").astype(int)

# 1 if POS contract is currently active in that month

POS_CASH_Balance["POS_COMPLETED_FLAG"] = (POS_CASH_Balance["NAME_CONTRACT_STATUS"] == "Completed").astype(int)
# 1 if POS contract is marked as completed in that month

# STEP 2 – One-hot encode contract status (month-level)

# This allows us to later compute the share of months spent in each status.

In [ ]:
pos_status_dummies = pd.get_dummies(
    POS_CASH_Balance["NAME_CONTRACT_STATUS"],
    prefix="POS_STATUS"
)
# One-hot encoded contract status, e.g. POS_STATUS_Active, POS_STATUS_Completed, POS_STATUS_Signed, etc.

POS_CASH_Balance = pd.concat([POS_CASH_Balance, pos_status_dummies], axis=1)


# STEP 3 – Define aggregation dictionary at customer level (SK_ID_CURR)
# I want to summarize POS loan behavior across all loans and all months per customer.

In [ ]:
agg_dict = {
    # Delinquency behavior (how often the customer has late months on POS loans)
    "POS_LATE_MONTH_FLAG": ["mean", "sum"],           # mean = share of late months, sum = total late months
    "POS_SERIOUS_LATE_MONTH_FLAG": ["mean", "sum"],   # serious overdue behavior

    # Instalment dynamics (how POS loan progresses over time)
    "CNT_INSTALMENT": ["mean", "max"],                # typical and maximum instalment count
    "CNT_INSTALMENT_FUTURE": ["mean", "min"],         # remaining instalments (lower is closer to completion)
    "POS_INSTALMENTS_PAID": ["mean", "max"],          # how many instalments are typically already paid
    "POS_REMAINING_RATIO": ["mean", "min", "max"],    # remaining fraction of POS loans
    "POS_PROGRESS_RATIO": ["mean", "max"],            # how far POS loans are progressed

    # Status flags
    "POS_ACTIVE_FLAG": ["mean", "sum"],               # share and count of months with active POS contracts
    "POS_COMPLETED_FLAG": ["mean", "sum"],            # share and count of months with completed POS contracts
}

# Add status dummy columns (distribution of contract statuses over time)
for col in pos_status_dummies.columns:
    agg_dict[col] = ["mean"]
    # mean of one-hot column = proportion of months in that specific contract status


# STEP 4 – Customer-level aggregation on SK_ID_CURR
# Each row in pos_agg will represent one customer (SK_ID_CURR)
# with summarized POS loan behavior across all POS loans and months.

In [ ]:
pos_agg = POS_CASH_Balance.groupby("SK_ID_CURR").agg(agg_dict)

# Flatten MultiIndex columns into single-level names
pos_agg.columns = [
    "POS_" + "_".join(col).upper() for col in pos_agg.columns
]
# Example: POS_POS_LATE_MONTH_FLAG_MEAN, POS_CNT_INSTALMENT_MAX, POS_POS_STATUS_ACTIVE_MEAN, etc.

pos_agg.reset_index(inplace=True)

print("Final POS_CASH customer-level shape:", pos_agg.shape)
# This table is now ready to be merged into the main application dataset.

# Merge

In [ ]:
# application_train = application_train.merge(pos_agg, on="SK_ID_CURR", how="left")

Processed 10M+ monthly POS loan snapshots and transformed them into customer-level features using SK_ID_CURR.

Validated the natural primary key as (SK_ID_PREV + MONTHS_BALANCE), ensuring one unique record per POS loan per month.

Engineered behavioural features such as delinquency flags (POS_SK_DPD, POS_SK_DPD_DEF), instalment progress, remaining instalment ratios, and contract activity indicators.

Used one-hot encoded contract status histories (Active, Completed, etc.) to measure the share of time customers spent in each loan status.

Aggregated all POS-level signals (mean, max, min, sum) to build customer-level POS risk profiles, ready to be merged into the main application training dataset.

In [ ]:
pos_agg.to_csv("pos_agg.csv", index=True)